# Improving Exercise Routine Adherence Through Interventions

[AIA's Vitality Program](https://www.aia.com.hk/en/our-products/aia-vitality.html) aims to encourages customers to follow a healthy lifestyle by engaging in exercises and leading an active lifestyle.
The program rewards customer through financial incentives by either increasing the insurance coverage or discounted insurance premiums.
However, in order to achieve long lasting effects it is crucial that customers form habits as well as keep these habits in the long run.

This project aims to improve habit formation and adherence of exercise routine through targeted and personalized interventions. 


## Example

Joe recently purchased a health insurance policy plan from AIA's Vitality Program.
In the first few days he started enthusiastically and met his daily step goal.
He also joined a gym and participated in a number of early morning classes.
Thus Joe started to form a exercise habit.y  

However, after a while his motivation to achieve his daily goals dropped.
This was mostly due to an increased work load at his current firm which required him to go to the office early.
Hence, making his frequent morning visits to the gym impossible.

## Intervention

Through Joe's historical exercise data we can detect changes in his exercise routines and formulate adequate interventions.
The here proposed project aims to use historical data of past exercise behavior to predict future behavior.
These prediction can then be used to compare the expected amount of exercise, given past behavior, to the actual amount of exercise.

In the example of Joe, we would predict a high amount of exercises in the morning, given his past behavior but actually observed hardly any physical activities due to his recent change in behavior.
This puts his exercise goals in danger and requires the need for an intervention to retain a persistent exercise habit.
These intervention could include:
- A simple notification with suggestions on how to exercise during lunch break
- A notification showing a list of evening classes at Joe's gym
- A notification containing a list of running routes near Joe's place of residence

Similarly, we can also look at weekly or monthly discrepancies between prediction and actual behavior in other variables such as weight or sleep.
Indeed, a change in sleep rhythm could potentially indicate an increase in stress levels and could have serious health consequences.
Targeted interventions to address these abnormality in sleep cycles could include bed-time notifications, or health questionnaires regarding coffee consumption as well as others.

# Demo

The here presented demo aims to provide a proof-of-concept (POC).
Please feel free to use the dropdown menu to select area's of interest, customer id and variable of interest.
The plot will then show past behavior as well as predicted future behavior. The shaded area within the plot indicates the 80% confidence interval of the prediction.
However, due to the limited data not all subjects have a prediction.

In [1]:
import httpx
import pandas as pd
from datetime import datetime
import json
import ipywidgets as widgets
from typing import List, Dict
import pandas as pd
from ipywidgets import interact, Dropdown
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
class DataRequest(object):
    
    def __init__(self, base_url: str):
        self._client = httpx.Client(base_url=base_url)
        self._data_cache = {}
    
    def _get(self, end_point: str):
        _hash = hash(end_point)
        if _hash in self._data_cache.keys():
            return self._data_cache[_hash]
        out = self._client.get(end_point).json()
        self._data_cache[_hash] = out
        return out
    
    def _post(self, end_point: str, data: Dict):
        _hash = hash(end_point) + hash(json.dumps(data, sort_keys=True))
        if _hash in self._data_cache.keys():
            return self._data_cache[_hash]
        out = self._client.post(end_point, json=data).json()
        self._data_cache[_hash] = out
        return out
    
    def get_tables(self):
        url = "table"
        return self._get(url)
    
    def get_users(self, table):
        url = f"user/table/{table}"
        return self._get(url)
    
    def get_data(self, user: str, num_items: int, table: str):
        url = "/user/data"
        req_body = {"user_id": user, "num_items": num_items, "table": table}
        output = self._post(url, req_body)
        return output
    
    def get_prediction(self, user: str, table: str):
        url = f"/user/prediction/{table}/{user}"
        output = self._get(url)
        return output
    
    def get_variable_names(self, table: str):
        url = f"/table/{table}"
        output = self._get(url)
        return output
    
    def get_data_prediction(self, user: str, num_items: int, table: str):
        data = self.get_data(user, num_items, table)
        pred_data = self.get_prediction(user, table)
        data_pd_ready = [{k: v for k, v in d.items() if k not in ("field_name", "user")} for d in data['data']]
        pred_data_pd_ready = [{k: v for k, v in d.items() if k not in ("field_name", "user")} for d in pred_data['data']]
        dt = pd.concat([pd.DataFrame(data_pd_ready), pd.DataFrame(pred_data_pd_ready)], axis=0)
        return dt.reset_index()

In [11]:
BASE_URL = "https://k2wnk7fz02.execute-api.ap-southeast-1.amazonaws.com/api/"

In [12]:
client = DataRequest(BASE_URL)

In [13]:
tables = client.get_tables()
first_user_options = client.get_users(tables[0])
first_variable_table = client.get_variable_names(tables[0])

In [15]:
# client.get_data_prediction(first_user_options[1], 24, tables[0])

In [17]:
output = widgets.Output()

def dropdown_eventhandler(change):
    determine(dropdown.value)

users_tables = {}
for tab in tables:
    users_tables[tab] = client.get_users(tab)

def determine(x):
    dropdown_dep.options = users_tables[x]
    
def figure_eventhandler_user(change):
    user = change.new
    plotting(user, dropdown.value, dropdown_var.value)
    display(output)

def variable_table(change):
    table_name = change.new
    dropdown_var.options = client.get_variable_names(table_name)


dropdown = Dropdown(description="Table:", options=tables)
dropdown_dep = Dropdown(description="User:", options=first_user_options)
dropdown_var = Dropdown(description="Variable:", options=first_variable_table)
dropdown.observe(dropdown_eventhandler, names='value')
dropdown.observe(variable_table, names='value')

button = widgets.Button(description="Get Predictions")

display(dropdown, dropdown_dep, dropdown_var, button, output)


def on_button_clicked(b):
    plotting(dropdown_dep.value, dropdown.value, dropdown_var.value)
    display(output)

button.on_click(on_button_clicked)


def plotting(user, table, variable, scope=24):
    dt = client.get_data_prediction(user, scope, table)
    with output:
        clear_output()
        figure, ax = plt.subplots(figsize=(22, 6))
        x = pd.to_datetime(dt['datetime'], infer_datetime_format=True)
        values = dt[variable].dropna().values
        y = np.concatenate([values, dt[variable+"_mean"].values[len(values):]])
        ymax = np.concatenate([values, dt[variable+"_upper"].values[len(values):]])
        ymin = np.concatenate([values, dt[variable+"_lower"].values[len(values):]])
        ax.plot(x, y)
        ax.fill_between(x, ymin, ymax, alpha=0.2)
        plt.show(figure)

Dropdown(description='Table:', options=('dailySleep', 'dailyWeight', 'dailyActivity', 'hourlyActivity'), value…

Dropdown(description='User:', options=('2347167796', '4319703577', '4388161847', '4020332650', '2026352035', '…

Dropdown(description='Variable:', options=('TotalSleepRecords', 'TotalMinutesAsleep', 'TotalTimeInBed'), value…

Button(description='Get Predictions', style=ButtonStyle())

Output()

# Technical implementation of this demo as well as future plans

Currently this demo uses 4 different components. AWS S3, AWS Lambda as well as a compute instance for model development.
The full architecture is outlined here:
![](demo_architecture.png)

This architecture, while potentially slightly to complicated to the purpose this demo aims to achieve, allows for easy extensions.
In particular, the lambda back-end allow to quickly change and switch out data sources. For instance, it might be more desirable to move from batch prediction in its current form, to online prediction. Thus allowing faster and more immediate responses. Indeed, given more time and resources one could implement a more comprehensive architecture:

![](proposed_architecture.png)